# Evaluation

Model: saved_model2 (Trained with 49,000 images)<br>
Input files: "working/test_df.csv"<br>
Output files: "working/df_evaluation_model2.csv"

In [1]:
import numpy as np
import pandas as pd
import os
import math
from glob import glob

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras import backend as K
from keras.optimizers import Adamax

import sklearn.metrics
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, classification_report

import PIL
from PIL import Image

import skimage
from skimage.io import imread
from skimage.color import rgb2lab, lab2rgb
from skimage.segmentation._quickshift import quickshift

import cv2

import shutil
from tqdm import tqdm
import albumentations as A

In [4]:
root = "d:/ham/"

df_path = root + "working/test_df.csv"
df = pd.read_csv(df_path)
df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,filepaths,cell_type,labels,ita,ita labels
0,HAM_0000827,ISIC_0031443,df,consensus,60.0,male,upper extremity,d:/ham/ham/HAM10000_images_part_2/ISIC_0031443...,Dermatofibroma,6,71.935040,1
1,HAM_0002211,ISIC_0026862,nv,follow_up,45.0,female,abdomen,d:/ham/ham/HAM10000_images_part_1/ISIC_0026862...,Melanocytic nevi,0,47.180616,1
2,HAM_0000529,ISIC_0030540,nv,follow_up,45.0,female,upper extremity,d:/ham/ham/HAM10000_images_part_2/ISIC_0030540...,Melanocytic nevi,0,2.025114,5
3,HAM_0004853,ISIC_0030333,mel,histo,80.0,male,upper extremity,d:/ham/ham/HAM10000_images_part_2/ISIC_0030333...,dermatofibroma,1,56.812119,1
4,HAM_0001902,ISIC_0026480,nv,consensus,5.0,female,lower extremity,d:/ham/ham/HAM10000_images_part_1/ISIC_0026480...,Melanocytic nevi,0,72.575822,1


In [5]:
# Labels should be string type
df["labels"] = df['labels'].apply(lambda x: str(x))

In [6]:
# check image size
cnt = 0
for f in df["filepaths"]:
    if cnt < 10:
        with Image.open(f) as img:
            aaa = img.size
        print(aaa)
    cnt = cnt + 1

(600, 450)
(600, 450)
(600, 450)
(600, 450)
(600, 450)
(600, 450)
(600, 450)
(600, 450)
(600, 450)
(600, 450)


In [7]:
from keras.preprocessing.image import ImageDataGenerator

img_size = (600,450)
batch_size=16
ycol='labels'

trgen=ImageDataGenerator(horizontal_flip=True)
t_and_v_gen=ImageDataGenerator()

def make_test_generator(xcol):
    length=len(df)
    test_batch_size=sorted([int(length/n) for n in range(1,length+1) if length % n ==0 and length/n<=80],reverse=True)[0]
    test_steps=int(length/test_batch_size)
    #print(length)
    msg='{0:70s} for test generator'.format(' ')
    print(msg, '\r', end='') # prints over on the same line
    test_gen=t_and_v_gen.flow_from_dataframe(df, x_col=xcol, y_col=ycol, target_size=img_size,
                                   class_mode='categorical', color_mode='rgb', shuffle=False, batch_size=test_batch_size)
    return test_gen

test_gen = make_test_generator("filepaths")

Found 1002 validated image filenames belonging to 7 classes.           for test generator 


In [8]:
def F1_score(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [9]:
def print_in_color(txt_msg,fore_tupple=(0,255,255),back_tupple=(100,100,100)):
    #prints the text_msg in the foreground color specified by fore_tupple with the background specified by back_tupple
    #text_msg is the text, fore_tupple is foregroud color tupple (r,g,b), back_tupple is background tupple (r,g,b)
    # default parameter print in cyan foreground and gray background
    rf,gf,bf=fore_tupple
    rb,gb,bb=back_tupple
    msg='{0}' + txt_msg
    mat='\33[38;2;' + str(rf) +';' + str(gf) + ';' + str(bf) + ';48;2;' + str(rb) + ';' +str(gb) + ';' + str(bb) +'m'
    print(msg .format(mat), flush=True)
    print('\33[0m', flush=True) # returns default print color to back to black
    return

In [11]:
# model load
model_path = root + "working/saved_model2/"
model = tf.keras.models.load_model(model_path, custom_objects={'F1_score': F1_score})

# Needs to compile again as for lr setting
lr=.001
model.compile(Adamax(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy', F1_score, 'AUC'])
model.summary()





Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 600, 450, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 600, 450, 3)          0         ['input_1[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 600, 450, 3)          7         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 rescaling_1 (Rescaling)     (None, 600, 450, 3)          0         ['normalization[0][0]'

In [12]:
# To check
model.layers[-1].get_weights()[0]

array([[ 0.00235935, -0.18162821, -0.12592748, ..., -0.13496386,
        -0.15794499, -0.18303472],
       [-0.96251297,  1.6716968 , -0.84361833, ..., -1.3162552 ,
        -1.90994   , -1.6251901 ],
       [-1.0693579 , -1.1946516 , -1.0779537 , ...,  1.829945  ,
        -1.7358441 , -1.4520962 ],
       ...,
       [-1.3265548 , -1.4702365 , -1.4958813 , ..., -1.5700049 ,
        -1.6845931 ,  2.169589  ],
       [-1.0516658 , -0.88379276,  1.7660949 , ..., -1.2377522 ,
        -1.5635567 , -1.5252115 ],
       [-0.9903722 ,  1.5742825 , -0.7556226 , ..., -0.97610706,
        -1.4696906 , -1.7495589 ]], dtype=float32)

In [13]:
def predictor(test_gen):
    y_pred= []
    error_list=[]
    error_pred_list = []
    y_true=test_gen.labels
    classes=list(test_gen.class_indices.keys())
    class_count=len(classes)
    errors=0
    preds=model.predict(test_gen, verbose=1)
    tests=len(preds)
    for i, p in enumerate(preds):
        pred_index=np.argmax(p)
        true_index=test_gen.labels[i]  # labels are integer values
        if pred_index != true_index: # a misclassification has occurred
            errors=errors + 1
            file=test_gen.filenames[i]
            error_list.append(file)
            error_class=classes[pred_index]
            error_pred_list.append(error_class)
        y_pred.append(pred_index)

    acc=( 1-errors/tests) * 100
    msg=f'there were {errors} errors in {tests} tests for an accuracy of {acc:6.2f}'
    print_in_color(msg, (0,255,255), (100,100,100)) # cyan foreground
    ypred=np.array(y_pred)
    ytrue=np.array(y_true)
    f1score=f1_score(ytrue, ypred, average='weighted')* 100
    clr = classification_report(y_true, y_pred, target_names=classes, digits= 4) # create classification report
    return errors, tests, error_list, error_pred_list, f1score, y_pred, y_true, clr

errors, tests, error_list, error_pred_list, f1score, y_pred, y_true, clr = predictor(test_gen)

167/167 [==============================] - 205s 1s/step
there were 162 errors in 1002 tests for an accuracy of  83.83



In [14]:
df["predictions"] = y_pred

In [16]:
df.to_csv(root + "working/df_evaluation_model2.csv", index=False)